In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Define Western Australia boundaries
# [112.8393° E, -35.1802° S, 129.0000° E, -13.7134° S]
WA_MIN_LON = 112.8393
WA_MAX_LON = 129.0000
WA_MIN_LAT = -35.1802
WA_MAX_LAT = -13.7134

# Load the grid coordinates
csv_path = r"C:\Users\ibian\Desktop\ClimAdapt\CMIP6\AGDC_grid_coordinates.csv"
df = pd.read_csv(csv_path)

print(f"Total coordinates in file: {len(df)}")
print(f"\nFirst few coordinates:")
print(df.head(10))
print(f"\nCoordinate ranges:")
print(f"Latitude: {df['latitude'].min()} to {df['latitude'].max()}")
print(f"Longitude: {df['longitude'].min()} to {df['longitude'].max()}")

In [ ]:
# Filter coordinates for Western Australia
wa_df = df[
    (df['longitude'] >= WA_MIN_LON) & 
    (df['longitude'] <= WA_MAX_LON) &
    (df['latitude'] >= WA_MIN_LAT) & 
    (df['latitude'] <= WA_MAX_LAT)
].copy()

print(f"Coordinates within Western Australia boundaries: {len(wa_df)}")
print(f"\nWA coordinate ranges:")
print(f"Latitude: {wa_df['latitude'].min()} to {wa_df['latitude'].max()}")
print(f"Longitude: {wa_df['longitude'].min()} to {wa_df['longitude'].max()}")

# Determine grid cell size (assuming regular grid)
lon_unique = sorted(wa_df['longitude'].unique())
lat_unique = sorted(wa_df['latitude'].unique())

if len(lon_unique) > 1:
    grid_size_lon = lon_unique[1] - lon_unique[0]
    print(f"\nGrid cell size (longitude): {grid_size_lon} degrees")
else:
    grid_size_lon = 0.05  # default assumption
    print(f"\nUsing default grid cell size (longitude): {grid_size_lon} degrees")

if len(lat_unique) > 1:
    grid_size_lat = lat_unique[1] - lat_unique[0]
    print(f"Grid cell size (latitude): {grid_size_lat} degrees")
else:
    grid_size_lat = 0.05  # default assumption
    print(f"Using default grid cell size (latitude): {grid_size_lat} degrees")

In [ ]:
def create_kml_grid_layer1(df, output_path, grid_size_lon, grid_size_lat, layer_name="WA Grid - Coordinates as Corners"):
    """
    Create KML file where coordinates represent the corners/edges of grid cells.
    Each coordinate becomes the top-left corner of a grid cell.
    """
    kml_header = '''<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
<Document>
    <name>''' + layer_name + '''</name>
    <Style id="gridStyle">
        <LineStyle>
            <color>ff0000ff</color>
            <width>1</width>
        </LineStyle>
        <PolyStyle>
            <color>400000ff</color>
            <fill>1</fill>
            <outline>1</outline>
        </PolyStyle>
    </Style>
'''
    
    kml_placemarks = ""
    
    for idx, row in df.iterrows():
        lon = row['longitude']
        lat = row['latitude']
        
        # Define cell corners (using coordinate as top-left corner)
        # Top-left, top-right, bottom-right, bottom-left
        corners = [
            (lon, lat),  # top-left
            (lon + grid_size_lon, lat),  # top-right
            (lon + grid_size_lon, lat - grid_size_lat),  # bottom-right
            (lon, lat - grid_size_lat),  # bottom-left
            (lon, lat)  # close polygon
        ]
        
        # Create placemark
        coordinates_str = " ".join([f"{lon},{lat},0" for lon, lat in corners])
        
        kml_placemarks += f'''    <Placemark>
        <name>Grid {idx+1}</name>
        <description>Center: ({lat:.4f}, {lon:.4f})</description>
        <styleUrl>#gridStyle</styleUrl>
        <Polygon>
            <outerBoundaryIs>
                <LinearRing>
                    <coordinates>{coordinates_str}</coordinates>
                </LinearRing>
            </outerBoundaryIs>
        </Polygon>
    </Placemark>
'''
    
    kml_footer = '''</Document>
</kml>'''
    
    kml_content = kml_header + kml_placemarks + kml_footer
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(kml_content)
    
    print(f"KML file saved to: {output_path}")
    print(f"Total grid cells: {len(df)}")

In [ ]:
def create_kml_grid_layer2(df, output_path, grid_size_lon, grid_size_lat, layer_name="WA Grid - Coordinates as Centers"):
    """
    Create KML file where coordinates represent the center points of grid cells.
    Each grid cell is centered on the coordinate.
    """
    kml_header = '''<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
<Document>
    <name>''' + layer_name + '''</name>
    <Style id="gridStyle">
        <LineStyle>
            <color>ff00ff00</color>
            <width>1</width>
        </LineStyle>
        <PolyStyle>
            <color>4000ff00</color>
            <fill>1</fill>
            <outline>1</outline>
        </PolyStyle>
    </Style>
'''
    
    kml_placemarks = ""
    half_lon = grid_size_lon / 2
    half_lat = grid_size_lat / 2
    
    for idx, row in df.iterrows():
        lon_center = row['longitude']
        lat_center = row['latitude']
        
        # Define cell corners (using coordinate as center)
        # Top-left, top-right, bottom-right, bottom-left
        corners = [
            (lon_center - half_lon, lat_center + half_lat),  # top-left
            (lon_center + half_lon, lat_center + half_lat),  # top-right
            (lon_center + half_lon, lat_center - half_lat),  # bottom-right
            (lon_center - half_lon, lat_center - half_lat),  # bottom-left
            (lon_center - half_lon, lat_center + half_lat)  # close polygon
        ]
        
        # Create placemark
        coordinates_str = " ".join([f"{lon},{lat},0" for lon, lat in corners])
        
        kml_placemarks += f'''    <Placemark>
        <name>Grid {idx+1}</name>
        <description>Center: ({lat_center:.4f}, {lon_center:.4f})</description>
        <styleUrl>#gridStyle</styleUrl>
        <Point>
            <coordinates>{lon_center},{lat_center},0</coordinates>
        </Point>
        <Polygon>
            <outerBoundaryIs>
                <LinearRing>
                    <coordinates>{coordinates_str}</coordinates>
                </LinearRing>
            </outerBoundaryIs>
        </Polygon>
    </Placemark>
'''
    
    kml_footer = '''</Document>
</kml>'''
    
    kml_content = kml_header + kml_placemarks + kml_footer
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(kml_content)
    
    print(f"KML file saved to: {output_path}")
    print(f"Total grid cells: {len(df)}")

In [ ]:
# Generate Layer 1: Grid with coordinates as grid cell corners/edges
output_path_layer1 = "WA_Grid_Layer1_Corners.kml"
create_kml_grid_layer1(wa_df, output_path_layer1, grid_size_lon, grid_size_lat, 
                       "Western Australia Grid - Coordinates as Cell Corners")

In [ ]:
# Generate Layer 2: Grid with coordinates as center points
output_path_layer2 = "WA_Grid_Layer2_Centers.kml"
create_kml_grid_layer2(wa_df, output_path_layer2, grid_size_lon, grid_size_lat,
                       "Western Australia Grid - Coordinates as Cell Centers")

## Summary

Two KML files have been generated:

1. **WA_Grid_Layer1_Corners.kml**: Grid where coordinates represent the top-left corners of grid cells
   - Style: Blue outlines
   - Coordinates from CSV are used as the top-left corner of each grid cell

2. **WA_Grid_Layer2_Centers.kml**: Grid where coordinates represent the center points of grid cells
   - Style: Green outlines
   - Coordinates from CSV are used as the center of each grid cell
   - Also includes a Point marker at each center coordinate

Both files are ready to be imported into Google Earth Pro.